# Sentiment Analysis
It could be interesting to correlate the greater public's interest into social good projects expressed on social media platforms such as Twitter and Facebook, or news platforms, with the actual investments that are done. 
To add the public opinion part, sentiment analysis was done on tweet from 2017. In fact, sentiment analysis is contextual mining of text which identifies and extracts subjective information in source material, and helping a business to understand the social sentiment of their brand, product or service while monitoring online conversations. In this part of the data story, the text classifier process tool that analyses an incoming message and tells whether the underlying sentiment is positive, negative our neutral will be describe.

 To do that, several questions needed to be answered. 
    - How to make a ranking of “public opinion” regarding a company?
    - What is the great public's opinion regarding the companies and investors ? 
    - Just capital ranking is one thing. This metrics is complex and tries to take into account a high number of variables. But it just capital score correlated with public opinion ? It means that we will try to understand the public’s opinion regarding the companies that held many investments in social good versus those who do not invest in social good?
 -SI JA I LE TEMPS CETTE NUIT IS A CORELATION BETWEEN THESES SCORES AND GEOGRAPHICAL INFORMATION EXISTING ? VOIR AVEC DAVE.


# Import

In [41]:
import pandas as pd
import numpy as np
import pickle
import os
from translate import translator
from googletrans import Translator
import plotly.plotly as py
import plotly.graph_objs as go
import re
import networkx as nx
from operator import itemgetter
import matplotlib.pyplot as plt
import collections
import fastText
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from networkx.algorithms.community.centrality import girvan_newman
import itertools
from community import community_louvain
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# General functions

In [6]:
class MacOSFile(object):
    '''
    This function allows the saving and the loading of huge files.
    '''

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size


def pickle_dump(obj, file_path):
    '''
    This function allows the saving of huge files to pickle.
    '''
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)

def pickle_load(file_path):
    '''
    This function allows the loading of huge files to pickle.
    '''
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [4]:
def clean_text(text):
    """
    Applies some pre-processing to clean text data.
    
    In particular:
    - lowers the string
    - removes the character [']
    - replaces punctuation characters with spaces

    """
    
    text = text.lower()

    text = re.sub(r"\'", "", text)  # remove the character [']

    # removing the punctuation
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    split = " "

    if isinstance(text, str):
        translate_map = dict((ord(c), str(split)) for c in filters)
        text = text.translate(translate_map)
    elif len(split) == 1:
        translate_map = maketrans(filters, split * len(filters))
        text = text.translate(translate_map)
    else:
        for c in filters:
            text = text.replace(c, split)
    return text

In [5]:
def translate(tweet):
    """
    This function allows the translation of a string to english language
    Arg :
        -tweet : String corresponding to the string we want to translate.
    """
    tw_translated = translator('en', tweet)
    return tw_translated

def translate(tweet,language):
    '''
    Other translate function using google API.
    '''
    translator = Translator()
    tw_inggris = translator.translate(tweet, src=language, dest='en')

In [5]:
def load_files(filepath, extension):
        '''
        This function allows the loading of all the files in a folder corresponding to the Data we want to use.
        Argument :
            - filepath : String corresponding to the folder where we want to select all the files with type_of_file and extension string in their names.
            - extension : String corresponding to the extension of the files we want to select and use. Should be '.p' or '.txt'.
        Return :
            - files : list of the files name order alphabetically that we will load.
        '''
        # Get all the files with the correponding type of file name:
        all_files = os.listdir(filepath)
        files = list()
        for file in all_files:
            if (extension in file):
                files.append(file)
        # Put them in alphabetical order:
        files = sorted(files)
        return files

In [6]:
def load_files_type(filepath, extension, type_of_file):
        '''
        This function allows the loading of all the files in a folder corresponding to the Data we want to use.
        Argument :
            - filepath : String corresponding to the folder where we want to select all the files with type_of_file and extension string in their names.
            - extension : String corresponding to the extension of the files we want to select and use. Should be '.p' or '.txt'.
            - type_of_file : String corresponding to the type of data we want to load. Should be 'Booking' or 'BOL'
        Return :
            - files : list of the files name order alphabetically that we will load.
        '''
        # Get all the files with the correponding type of file name:
        all_files = os.listdir(filepath)
        files = list()
        for file in all_files:
            if (extension in file)&(type_of_file in file):
                files.append(file)
        # Put them in alphabetical order:
        files = sorted(files)
        return files

In [1]:
def filter_english_tweet(df):
    '''
    This function allows the filtering of the tweet which are not written in english.
    Arg :
        - df : Raw dataframe from the tweet dataset we want to filter.
    Return :
        - df : Input dataframe filtered.
    '''
    df = df[df.lang=='en']
    df = df.reset_index()
    return df

In [ ]:
def decompress_json(archive_path, outfile_path):
    outfile_path = outfile_path[:-4]
    with open(archive_path, 'rb') as source, open(outfile_path, 'wb') as dest:
        dest.write(bz2.decompress(source.read()))

# Dataset

## Load json data export from the clusters :


In [9]:
import bz2
for i in np.arange(12):
    for j in ['01','11','21']:
        list_df = []
        for k in range(24):
            files = []
            if k<10:
                files = load_files(f'../../Desktop/0{i}/{j}/0{k}/', '.json')
                for file in files:
                    if file.find('bz2')==-1:
                        list_df.append(pd.read_json(f'../../Desktop/0{i}/{j}/0{k}/{file}'))
            else:
                files = load_files(f'../../Desktop/0{i}/{j}/{k}/', '.json')
                for file in files:
                    if file.find('bz2')==-1:
                        list_df.append(pd.read_json(f'../../Desktop/0{i}/{j}/{k}/{file}'))

        if len(list_df)!=0:
            df = pd.concat(list_df)
            pickle_dump(df,f'../../data_twitter/load_tweet_2017_{i}_{j}.p')

KKKKK 0


FileNotFoundError: [Errno 2] No such file or directory: '../../Desktop/06/21/00/'

# Preprocess

First of all, output of the last part where taken and mix with a new dataset which is the Twitter dataset given in the course. It contains 1% of the tweets of 2017 and it’s store on the ADA cluster. The documentation of Twitter API was analyzed to understand how to use this database.The group decided to work on a selected part of the entire dataset. In fact, our goal is to score the public’s opinion of compagnies thanks to a sentiment analysis done over tweets of 2017. To do that we decide to take 3 days per month written in english. English tweets were filter as our analysis is made on Americans compagnies. We thus care more about the public opinion of English speaking population as they are more impacted by theses compagnies. For each month, we take always the same days : the first one, the 11th one and the 21th. Theses two filter process allows us to work on a twitter dataset with the following attributes : 
ADD THE DESCRIPTION OF THE DATASET : Nan et … FAIRE PANDAS .DESCRIBE



Then, to make it easy to use, we create a dictionary containing all the tweets and the tags cleaned. ‘Cleaned’ means that for each string, we lowers the string, we removes the character [‘], and we replaces punctuation characters with spaces.
Then, the creation of a dictionary of all company and investor was be done and finally, the company or investor cleaned name was used as key of the dictionary, grouping all the tweets about it together.Tweets were filter using a easy rule : a tweet concerned a company if the name of the company is present in the tweet text or the the tweet tags. All this process gives us a dates with the following specificity : WRITE ALLL SPECIFICITIES
PEUT ETRE 



## Create a dictionary containing each tweet with the tag

In [ ]:
def dict_tweet_function(df):
    list_hashtags = []
    dict_tot = dict()
    for i in range(df.text.index.max()):
        print(f'{i} over {df.text.index.max()}')
        dict_tmp = dict()
        if (type(df.text[i])==str):
            if (len(df.entities[i]['hashtags'])!=0):
                dict_tmp['hashtags'] = df.entities[i]['hashtags'][0]['text']
                dict_tmp['text'] = df.text[i]
            else :
                dict_tmp['hashtags'] = df.entities[i]['hashtags']
                dict_tmp['text'] = df.text[i]
            dict_tot[(i,-1)]=dict_tmp
        elif type(df.text[i])==pd.core.series.Series: 
            for j in range(len(df.text[i])): 
                dict_tmp = dict()
                if (type(df.text[i].reset_index().text[j])==str):
                    if len(df.entities[i].reset_index().entities[j]['hashtags'])!=0:
                        dict_tmp['hashtags'] = df.entities[i].reset_index().entities[j]['hashtags'][0]['text']
                        dict_tmp['text'] = df.text[i].reset_index()['text'][j]
                    else : 
                        dict_tmp['hashtags'] = df.entities[i].reset_index().entities[j]['hashtags']
                        dict_tmp['text'] = df.text[i].reset_index()['text'][j]
                    dict_tot[(i,j)]=dict_tmp
        pickle_dump(dict_tot, f'../../data_twitter/dict/dict_all_tweet_2017_1_01.p')
    return dict_tot

df = pickle_load(f'../../data_twitter/load_tweet_2017_1_01.p')
tmp = dict_tweet_function(df)

load_tweet_2017_1_01.p


In [ ]:
dict_tot = pickle.load(open('dictionary_with_all_tweets_and_hashtags.p', 'rb'))

## Merge several process files done in a segmented way

In [ ]:
files = load_files(f'../../data_twitter/dict/', '.json')
df = Dataframe()
for file in files:
    df = pd.concat(df, pickle_load(f'../../data_twitter/dict/{file}'))

## Load the list of investor and compagny we are working on
We just take compagnies/investor present in 2017 as our sentiment anlysis is done on this period.

In [9]:
df = pd.read_csv('../../Desktop/13f_results_13-18Q3_merged.csv')
df['date'] = pd.to_datetime(df['date'])
df['Year'] = df['date'].dt.year
df = df[df.Year == 2017]
df = df.reset_index()
pickle_dump(df.investor.unique(), 'investor.p')
pickle_dump(df.name.unique(), 'compagny.p')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.



writing total_bytes=75136...
writing bytes [0, 75136)... done.
writing total_bytes=3768...
writing bytes [0, 3768)... done.


Here we clean the name of the compagny/investor. In particular:
    - lowers the string
    - removes the special character 
    - replaces punctuation characters with spaces
Then we split the string and select the longest word and take as name of the compagny the longest word one plus one word before. In fact, after analysis the dataset, this methods gives the best results compare to other trials.

In [193]:
tmp = pd.read_csv('../../Desktop/13f_results_13-18Q3_merged.csv')
compagny = tmp['name'].unique()
investor = tmp['investor'].unique()

# Clean the name : 
for i in range(len(compagny)):
    index = clean_text(compagny[i]).split().index(max(clean_text(compagny[i]).split(), key=len))
    if index > 0:
        compagny[i] = f'{clean_text(compagny[i]).split()[index_name-1]} {clean_text(compagny[i]).split()[index_name]}'
    else :
        compagny[i] = clean_text(compagny[i]).split()[0]
        
for i in range(len(investor)):
    index = clean_text(investor[i]).split().index(max(clean_text(investor[i]).split(), key=len))
    if index > 1:
        investor[i] = f'{clean_text(investor[i]).split()[index_name-2]} {clean_text(investor[i]).split()[index_name-1]} {clean_text(investor[i]).split()[index_name]}'
    elif index > 0:
        investor[i] = f'{clean_text(investor[i]).split()[index_name-1]} {clean_text(investor[i]).split()[index_name]}'
    else :
        investor[i] = clean_text(investor[i]).split()[0]

In [194]:
print(len(compagny))
print(len(investor))

217
5146


## Create a dictionary using the compagny or investor as key grouping all the tweets about it
We consider that the tweet concerned a compagny/investor is there is his name in the tweet or the hashtag

In [200]:
def tweet_per_category(list_of_names, dict_tot):
    dict_per_category = dict()
    i = 0
    for name in list_of_names :
        print(i)
        list_tweet = []
        for key in list(dict_tot.keys()) :
            if (str(dict_tot[key]['hashtags']).lower().find(name.lower()) != -1) | (dict_tot[key]['text'].lower().find(name.lower()) != -1):
                list_tweet.append(dict_tot[key]['text'])
        dict_per_category[name] = list_tweet
        i=i+1
    return dict_per_category

In [201]:
dict_tot = pickle_load(f'../../data_twitter/dict/dict_all_tweet_2017_1_01.p')
dict_per_compagny = tweet_per_category(compagny, dict_tot)
pickle_dump(dict_per_compagny, 'dictionary_per_compagny_tweet.p')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
writing total_bytes=43064800...
writing bytes [0, 43064800)... done.


In [ ]:
dict_tot = pickle_load(f'../../data_twitter/dict/dict_all_tweet_2017_1_01.p')
dict_per_investor = tweet_per_category(investor, dict_tot)
pickle_dump(dict_per_investor, 'dictionary_per_investor_tweet.p')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
dict_per_compagny = pickle.load(open('dictionary_per_compagny_tweet.p', 'rb'))

In [ ]:
dict_per_investor = pickle.load(open('dictionary_per_investor_tweet.p', 'rb'))

# Method of sentiment analysis :

After this pre-processing part, sentiment analysis can begin. We used 2 different method. 
We apply theses two different method for each tweet and thus as output we have two different scores per tweet (Vader and Fastest). We do the mean of theses two scores. Then if there are several tweets, we take the mean. In fact, we want to penalized when we have extreme score.


## Make sentiment analysis for each tweet for each compagny/investors :
### Vader

The first one is Vader. VADER belongs to a type of sentiment analysis that is based on lexicons of sentiment-related words. In this approach, each of the words in the lexicon is rated as to whether it is positive or negative, and in many cases, how positive or negative. As you might have guessed, when VADER analyses a piece of text it checks to see if any of the words in the text are present in the lexicon. VADER produces four sentiment metrics from these word ratings, which you can see below. The first three, positive, neutral and negative, represent the proportion of the text that falls into those categories.  The final metric, the compound score, is the sum of all of the lexicon ratings which have been standardised to range between -1 and 1. Moreover, VADER doesn’t just do simple matching between the words in the text and in its lexicon. It also considers certain things about the way the words are written as well as their context. One of the things that VADER recognises is capitalisation, which increases the intensity of both positive and negative words. Another factor that increases the intensity of sentence sentiment is exclamation marks, with up to 3 exclamation marks adding additional positive or negative intensity. VADER also takes into account what happens when modifying words are present in front of a sentiment term. Finally, VADER also handles changes in a sentence’s sentiment intensity when it contains ‘but’. Essentially, the rule is that the sentiments expressed both before and after the ‘but’ are taken into consideration, but the sentiment afterwards is weighted more heavily than that before.

This library takes into account :

- positive sentence example
- punctuation emphasis handled correctly (sentiment intensity adjusted)
- booster words handled correctly (sentiment intensity adjusted)
- emphasis for ALLCAPS handled
- combination of signals - VADER appropriately adjusts intensity
- booster words & punctuation make this close to ceiling for score
- negation sentence example
- positive sentence
- negated negative sentence with contraction
- qualified positive sentence is handled correctly (intensity adjusted)
- mixed negation sentence
- negative slang with capitalization emphasis
- mixed sentiment example with slang and constrastive conjunction "but"
- emoticons handled
- emojis handled
- Capitalized negation

In [ ]:
def vader_analysis(dict_per_):
    i = 0
    analyzer = SentimentIntensityAnalyzer()
    dict_score = dict()
    for key in list(dict_per_.keys()):
        print(f'{i} over {len(dict_per_.keys())}')
        neg, pos, neu, compound, tmp_dict = [], [], [], [], dict()
        for sentence in dict_per_[key]:
            vs = analyzer.polarity_scores(sentence)
            compound.append(vs['compound'])
        tmp_dict['compound'] = np.mean(compound)
        dict_score[key] = tmp_dict 
        i=i+1
        
    return dict_score

In [ ]:
output = vader_analysis(dict_per_compagny)
pickle.dump(output, open('dictionary_per_compagny_score_vader.p', 'wb'))

In [ ]:
df = pickle.load(open('dictionary_per_compagny_score_vader.p','rb'))

In [ ]:
output = vader_analysis(dict_per_investor)
pickle.dump(output, open('dictionary_per_investor_score_vader.p', 'wb'))

### FastText: 

The second one is FastText. To be efficient on datasets with very large number of categories, it uses a hierarchical classifier instead of a flat structure, in which the different categories are organized in a tree (think binary tree instead of list). This reduces the time complexities of training and testing text classifiers from linear to logarithmic with respect to the number of classes. FastText also exploits the fact that classes are imbalanced (some classes appearing more often than other) by using the Huffman algorithm to build the tree used to represent categories. The depth in the tree of very frequent categories is therefore smaller than for infrequent ones, leading to further computational efficiency.FastText also represents a text by a low dimensional vector, which is obtained by summing vectors corresponding to the words appearing in the text. In fastText, a low dimensional vector is associated to each word of the vocabulary. This hidden representation is shared across all classifiers for different categories, allowing information about words learned for one category to be used by other categories. These kind of representations, called bag of words, ignore word order.

In [ ]:
MODEL_DIR_PATH = "../../data_twitter/"

# Load a pretrained model for classification
model = fastText.load_model(os.path.join(MODEL_DIR_PATH, "amazon_review_full.bin"))

maketrans = str.maketrans

In [ ]:
predict_sentiment = lambda s: model.predict(clean_text(s))

In [ ]:
def fastext_prediction(dict_per_):
    dict_score = dict()
    i=0
    for key in list(dict_per_.keys()):
        print(f'{i} over {len(dict_per_.keys())}')
        label, confidence, tmp_dict = [], [], dict()
        for sentence in dict_per_[key]:
            res = predict_sentiment(sentence)
            label.append(int(res[0][0][9]))
            confidence.append(res[1][0])
        tmp_dict['label'] = np.mean(label)
        tmp_dict['confidence'] = np.mean(confidence)
        dict_score[key] = tmp_dict 
        i=i+1

    return dict_score

In [ ]:
output = fastext_prediction(dict_per_compagny)
pickle.dump(output, open('dictionary_per_compagny_score_fasttext.p', 'wb'))

In [ ]:
output = fastext_prediction(dict_per_investor)
pickle.dump(output, open('dictionary_per_investor_score_fasttext.p', 'wb'))

# Visualisation Part

### Preprocessing : Import scores and plot the distribution  :

In this part, we are going pre-process the data to have a good format to allows the usage of Gephi to make some grah plot. To do that we will first create dataframe per type of error and per category (investor/compagny). Then, a third scores will be created : the mean of the both normalized scores. We normalized it to give the same importance to each metrics. Then, merging of compagny and investor data will be done. The categorical features is kept by creating a category features with two different string : 'investor' and 'compagny'. To conclude this preprocessing part, node and edge csv files will be created to allows a easy usage of Gephi tool. A correlation dataframe will also be created to allow the creation of a scatter plot. This dataframe will contain Just Capital score and sentiment analysis scores for all compagnies/investors.

In this first part, dataframe creation, normalization of fasttext score and merging both metrics will be done.

### Compagnies scores

#### Vader

In [ ]:
df = pickle.load(open('dictionary_per_compagny_score_vader.p','rb'))
dataframe_compagny = pd.DataFrame()

In [ ]:
list_key, list_score = [],[]
for key in df.keys():
    list_key.append(key)
    list_score.append(df[key]['compound'])
dataframe_compagny['compagny'] = list_key
dataframe_compagny['Vader_score'] = list_score

In [ ]:
# Plot
data = [go.Histogram(x=dataframe_compagny.Vader_score)]
py.iplot(data, filename='Vader Score histogram for compagnies')

#### FastText Score

In [ ]:
df = pickle.load(open('dictionary_per_compagny_score_fastext.p','rb'))

In [ ]:
import sklearn.preprocessing
list_key, list_score, list_confidence = [],[],[]
for key in df.keys():
    list_key.append(key)
    list_score.append(df[key]['label']) 
    list_confidence.append(df[key]['confidence'])
dataframe_compagny['compagny'] = list_key
dataframe_compagny['FastText_score'] = list_score
dataframe_compagny['FastText_confidence'] = list_confidence
# We normalize the score
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(dataframe_compagny[['FastText_score','FastText_confidence']].fillna(0))
dataframe_compagny[['FastText_score','FastText_confidence']] = scaler.transform(dataframe_compagny[['FastText_score','FastText_confidence']].fillna(0))



In [ ]:
#Plot
score = go.Histogram(x=dataframe_compagny.FastText_score, opacity=0.8)
confidence = go.Histogram(x=dataframe_compagny.FastText_confidence, opacity=0.5)

data = [score, confidence]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='FastTest Score and confidence histogram for compagnies')

###  Investors scores :

#### Vader :

In [ ]:
df = pickle.load(open('dictionary_per_investor_score_vader.p','rb'))
dataframe_investor = pd.DataFrame()

In [ ]:
list_key, list_score = [],[]
for key in df.keys():
    list_key.append(key)
    list_score.append(df[key]['compound'])
dataframe_investor['compagny'] = list_key
dataframe_investor['Vader_score'] = list_score

In [ ]:
# Plot
data = [go.Histogram(x=dataframe_investor.Vader_score)]
py.iplot(data, filename='Vader Score histogram for compagnies')

#### FastText :

In [ ]:
df = pickle.load(open('dictionary_per_investor_score_fastext.p','rb'))

In [ ]:
list_key, list_score, list_confidence = [],[],[]
for key in df.keys():
    list_key.append(key)
    list_score.append(df[key]['label'])
    list_confidence.append(df[key]['confidence'])
dataframe_investor['compagny'] = list_key
dataframe_investor['FastText_score'] = list_score
dataframe_investor['FastText_confidence'] = list_confidence

# We normalize the score
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(dataframe_investor[['FastText_score','FastText_confidence']].fillna(0))
dataframe_investor[['FastText_score','FastText_confidence']] = scaler.transform(dataframe_investor[['FastText_score','FastText_confidence']].fillna(0))

In [ ]:
score = go.Histogram(x=dataframe_investor.FastText_score, opacity=0.75)
confidence = go.Histogram(x=dataframe_investor.FastText_confidence, opacity=0.75)

data = [score, confidence]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='FastTest Score and confidence histogram for compagnies')

In [ ]:
# Save intermediate steps 
pickle_dump(dataframe_investor,'dataframe_investor_both_scores.p')
pickle_dump(dataframe_compagny,'dataframe_compagny_both_scores.p')

### Distribution histogram for both scores

#### Compagnies

In [ ]:
ft_score = go.Histogram(x=dataframe_compagny.FastText_score, opacity=0.75)
vd_score = go.Histogram(x=dataframe_compagny.Vader_score, opacity=0.75)
confidence = go.Histogram(x=dataframe_compagny.FastText_confidence, opacity=0.75)

data = [ft_score, vd_score, confidence]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='Both Score histogram for compagnies')

#### Investors

In [ ]:
ft_score = go.Histogram(x=dataframe_investor.FastText_score, opacity=0.75)
vd_score = go.Histogram(x=dataframe_investor.Vader_score, opacity=0.75)
confidence = go.Histogram(x=dataframe_investor.FastText_confidence, opacity=0.75)

data = [ft_score, vd_score, confidence]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='Both Score histogram for compagnies')

### Merge investors and compagnies in the same dataframe
Doing this process will help us doing the following graphs.

In [ ]:
# We need this features for categories different categories in the futur plots.
dataframe_investor['category'] = 'investor'
dataframe_compagny['category'] = 'compagny'
dataframe_final_scores = pd.concat([dataframe_compagny, dataframe_investor], ignore_index = True)

In [ ]:
dataframe_final_scores.head()

In [ ]:
ft_score = go.Histogram(x=dataframe_final_scores.FastText_score, opacity=0.75)
vd_score = go.Histogram(x=dataframe_final_scores.Vader_score, opacity=0.75)
confidence = go.Histogram(x=dataframe_final_scores.FastText_confidence, opacity=0.75)

data = [ft_score, vd_score, confidence]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='Both Score histogram for compagnies')

### Merge both scores in a single one

In [ ]:
dataframe_final_scores['Scores_Total'] = dataframe_final_scores[['Vader_score','FastText_score']].mean(axis = 1)

In [ ]:
dataframe_final_scores.head()

In [ ]:
ft_score = go.Histogram(x=dataframe_final_scores.FastText_score, opacity=0.75)
vd_score = go.Histogram(x=dataframe_final_scores.Vader_score, opacity=0.75)
confidence = go.Histogram(x=dataframe_final_scores.FastText_confidence, opacity=0.30)
tot_score = go.Histogram(x=dataframe_final_scores.Scores_Total, opacity=0.75)

data = [ft_score, vd_score, confidence, tot_score]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='Both Score histogram for compagnies')

### Create node dataframe for the merge data with the 3 different type of scores

In [ ]:
# In this cell, we prepare the dataset to be use in Gephi to do beautiful graph plot
dataframe_final_scores.category = dataframe_final_scores.category.astype('category')
dataframe_final_scores = dataframe_final_scores[['compagny', 'Vader_score', 'FastText_score', 'FastText_confidence','Score_Total', 'category']]
dataframe_final_scores = dataframe_final_scores.rename(columns = {'compagny' : 'ID'}).reset_index().drop(['index'], axis =1)
dataframe_final_scores = dataframe_final_scores.set_index('ID')
dataframe_final_scores = dataframe_final_scores.groupby(dataframe_final_scores.index).sum()
dataframe_final_scores.head() ####attention dans rename

In [ ]:
dataframe_final_scores.isnull().mean()

In [ ]:
pickle_dump(dataframe_final_scores,'node_df.csv')

### Create the corresponding edge matrix

In [ ]:
# Load and clean load data and select data of interest (which means for year 2017)
df_edges = pd.read_csv('../../Desktop/13f_results_13-18Q3_merged.csv')
print(len(df_edges))
df_edges['date'] = pd.to_datetime(df_edges['date'])
df_edges['Year'] = df_edges['date'].dt.year
df_edges = df_edges[df_edges.Year == 2017]
df_edges = df_edges.reset_index()
print(len(df_edges))
# Select meaning full features
df_edges = df_edges[['investor','date','stock','name', 'industry','scores.overall']]
# Create the edges matrix by cleaning it
for i in range(len(df_edges)):
    index_name = clean_text(df_edges['name'][i]).split().index(max(clean_text(df_edges['name'][i]).split(), key=len))
    if index_name > 0:
        df_edges['name'][i] = f'{clean_text(df_edges.name[i]).split()[index_name-1]} {clean_text(df_edges.name[i]).split()[index_name]}'
    else :
        df_edges['name'][i] = clean_text(df_edges.name[i]).split()[0]
    
    index_investor = clean_text(df_edges['investor'][i]).split().index(max(clean_text(df_edges['investor'][i]).split(), key=len))
    elif index_investor > 1 :
        df_edges['investor'][i] = f'{clean_text(df_edges.investor[i]).split()[index_investor-2]} {clean_text(df_edges.investor[i]).split()[index_investor-1]} {clean_text(df_edges.investor[i]).split()[index_investor]}'
    elif index_investor > 0 :
        df_edges['investor'][i] = f'{clean_text(df_edges.investor[i]).split()[index_investor-1]} {clean_text(df_edges.investor[i]).split()[index_investor]}' 
    else :
        df_edges['investor'][i] = clean_text(df_edges.investor[i]).split()[0]
    

In [ ]:
df_edges = df_edges.rename(columns = {'investor':'Source', 'name':'Target'})
df_edges = df_edges.reset_index()[['Source','Target']]
df_edges = df_edges.set_index(['Source'])

In [ ]:
pickle_dump(df_edges,'edge_df.csv')

We use this new matrix to take just the node present in this edge matrix :

In [ ]:
A COMPLETER !!!!!! 
all_name
dataframe_final_scores = dataframe_final_scores[dataframe_final_scores.index.isin()]
pickle_dump(dataframe_final_scores,'node_df.csv')

### Create correlation dataframe to produce the following scatter plot

In [ ]:
score_capital = []
correlation = df_edges
correlation['scores_sentiment_compagny'] = 0.0
for i in range(correlation.shape[0]) :
    for j in range (dataframe_final_scores.shape[0]) :
        if correlation['name'][i] == dataframe_final_scores.index[j]:
            correlation['scores_sentiment_compagny'][i] = float(dataframe_final_scores.loc[dataframe_final_scores.index == dataframe_final_scores.index[j],'Interval'].values[0])

In [ ]:
correlation.to_pickle('correlation.p')

### Scatter plot to analyse correlation between Just Capital scores and sentiment analysis scores 

In [ ]:
# Create a trace
trace = go.Scatter(
    x = correlation['scores.overall'],
    y = correlation['scores_sentiment_compagny'],
    mode = 'markers'
)

data = [trace]

fig = dict(data=data)
py.iplot(fig, filename='Correlation between Sentiment Analysis scores and JustCapital Scores')

# Plots

### Graph plot exploration :

In [ ]:
print('There are {df_node.shape[0]} compagnies/investors linked with {df_edges.shape[0]} edges in total')

In [ ]:
# Put the edges information in the good format
link_graph = nx.from_pandas_edgelist(df_edges, 'Source', 'Target', edge_attr=None, create_using= nx.Graph())
# Set attributes
nx.set_node_attributes(link_graph, df_node['Vader_score'].to_dict(), 'Vader_score' )
nx.set_node_attributes(link_graph, df_node['FastText_score'].to_dict(), 'FastText_score' )
link_graph.node['caterpillar']

In [ ]:
# We connect the network
pos = nx.spring_layout(link_graph,k=0.2)
ec = nx.draw_networkx_edges(link_graph, pos, alpha=0.4)
nc = nx.draw_networkx_nodes(link_graph, pos, nodelist=link_graph.nodes(), node_color='g', cmap=plt.cm.jet, node_shape='.')
plt.axis('off')
plt.show()

This visualization is not a very pretty. That's why we installed Gephy for some better-looking plots.
Let's answer the classical question of a graph analysis.

In [ ]:
print("Network sparsity: %.4f" %nx.density(link_graph))
print(nx.is_connected(link_graph))
comp = list(nx.connected_components(link_graph))
print('The graph contains', len(comp), 'connected components')
largest_comp = max(comp, key=len)
percentage_lcc = len(largest_comp)/ link_graph.number_of_nodes() * 100
print('The largest component has ', len(largest_comp), 'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes')
path = nx.shortest_path(link_graph, source="caterpillar", target="hca")
print("Shortest path between caterpillar and hca:", path)

From visually inspecting the graph, we already saw that most of the nodes are in the largest component, which is often the case in a graph. That is why we call that a giant component

In [ ]:
# take the largest component and analyse its diameter = longest shortest path 
lcc_link_graph = link_graph.subgraph(largest_comp)
diameter = nx.diameter(lcc_link_graph)
print("The diameter of the largest component is", diameter, "which is the longest shortest path between any two compagnies in the giant component")

Let's plot a subgraph to see the form :

In [ ]:
subgraph_hca = link_graph.subgraph(['apple']+list(link_graph.neighbors('apple')))
nx.draw_spring(subgraph_hca, with_labels=True)

Which compagny/investors are the most important ?
There are many ways to detect important nodes in a graph, for example based on degree or betweeness centrality.

Let's begin with the degree : This view is that the more compagnies you are connected with, the more important you are. We will look at the degree distribution.

In [ ]:
degrees = dict(link_graph.degree(link_graph.nodes()))
sorted_degree = sorted(degrees.items(), key=itemgetter(1), reverse=True)

# And the top 5 most popular quakers are.. 
for compagny, degree in sorted_degree[:5]:
    print(compagny, ' who is', link_graph.node[compagny]['Score'], 'is linked', degree, 'compganies/investors')

In [ ]:
degree_seq = [d[1] for d in sorted_degree]
degreeCount = collections.Counter(degree_seq)
degreeCount = pd.DataFrame.from_dict( degreeCount, orient='index').reset_index()
fig = plt.figure()
ax = plt.gca()
ax.plot(degreeCount['index'], degreeCount[0], 'o', c='blue', markeredgecolor='none', markersize= 4)
plt.ylabel('Frequency')
plt.xlabel('Degree')
plt.title('Degree distribution for the investors/compagnies network')

Now betweeness : The other view is that the more shortest paths pass through a node, the more important it is.

In [ ]:
betweenness = nx.betweenness_centrality(link_graph) # Run betweenness centrality
# Assign each to an attribute in your network
nx.set_node_attributes(link_graph, betweenness, 'betweenness')
sorted_betweenness = sorted(betweenness.items(), key=itemgetter(1), reverse=True)

for quaker, bw in sorted_betweenness[:5]:
    print(quaker, ' who is', link_graph.node[quaker]['Score'], 'has betweeness: %.3f' %bw)

In [ ]:
# similar pattern
list_nodes =list(link_graph.nodes())
list_nodes.reverse()   # for showing the nodes with high betweeness centrality 
pos = nx.spring_layout(link_graph)
ec = nx.draw_networkx_edges(link_graph, pos, alpha=0.1)
nc = nx.draw_networkx_nodes(link_graph, pos, nodelist=list_nodes, node_color=[link_graph.nodes[n]["betweenness"] for n in list_nodes], 
                            with_labels=False, alpha=0.8, node_shape = '.')
plt.colorbar(nc)
plt.axis('off')
plt.show()

Try clustering : The Louvain method is a clustering algorithm and has become a standard algorithm in the data scientist toolbox.

It proceeds the other way around: initially every node is considered as a community. The communities are traversed, and for each community it is tested whether by joining it to a neighboring community, we can obtain a better clustering.

In [ ]:
# clustering
partition = community_louvain.best_partition(link_graph)
# add it as an attribute to the nodes
for n in link_graph.nodes:
    link_graph.nodes[n]["louvain"] = partition[n]

In [ ]:
# plot it out
pos = nx.spring_layout(link_graph,k=0.2)
ec = nx.draw_networkx_edges(link_graph, pos, alpha=0.2)
nc = nx.draw_networkx_nodes(link_graph, pos, nodelist=link_graph.nodes(), node_color=[link_graph.nodes[n]["louvain"] for n in link_graph.nodes], 
                            with_labels=False, node_size=100, cmap=plt.cm.jet)
plt.axis('off')
plt.show()


In [ ]:
cluster= partition['apple']
# Take all the nodes that belong to William's cluster
members_c = [q for q in link_graph.nodes if partition[q] == cluster]
# get info about these quakers
for compagny in members_c:
    print(compagny, ' who is', link_graph.node[compagny].keys())

To finish this analysis, we will calculate the Homophily which like correlation, but translated into graphs.

It means : How likely two compagnies that have the same attribute(just_capital_score or sentiment_analysis_score) are linked?

In [ ]:
# for categorical attributes
print (f'Homophily Vader_score :', nx.attribute_assortativity_coefficient(link_graph, 'Vader_score'))
# for categorical attributes
print (f'Homophily FastText_score :', nx.attribute_assortativity_coefficient(link_graph, 'FastText_score')
# for categorical attributes
print (f'Homophily Score_Total :', nx.attribute_assortativity_coefficient(link_graph, 'Score_Total')
# for categorical attributes
print (f'Homophily Just_Capital :', nx.attribute_assortativity_coefficient(link_graph, 'Just_Capital')

### Plots done with Gephi :

In [ ]:
A COMPLETER
Graph degré
Graph Vader
Graph Fastext
Graph merge
Graph just capital
tenté map

In [ ]:
rajouter endroit ou merge file
De plus faire comme d hab avec ceux qui ne sont pas relié et enlever ces noeud soit avec filtre soit en faisant un is in pour les noeud --> objectif baisser le nb de noeud.
problem category et re run